## Design and implementation

In [255]:
import pandas as pd
import numpy as np
import altair as alt
import geopandas as gpd

alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [256]:
collisions = pd.read_csv('../data/preprocessed-collisions-final.csv')

### Are accidents more frequent during weekdays or weekends? Is there any difference between before COVID-19 and after?

We need to distinguish between weekdays and weekends and also between the periods before and after COVID (2018 and 2020). Our initial approach involves organizing the data with the day of the week on the x-axis and aggregating the number of accidents for each day. This method should allow us to observe any significant differences between weekdays and weekends.

To compare the number of accidents before and after COVID, we propose using a paired bar chart. This visual representation will employ distinct colors for each period, providing a clear comparison between the two.

In [257]:
frequent_collisions = collisions[['CRASH_DATETIME', 'day_week', 'type_day']]
frequent_collisions.to_csv('../data/frequent-collisions.csv', index=False)

In [258]:
paired_bar_chart = alt.Chart(frequent_collisions).mark_bar().encode(
  x = alt.X('year:O', title = 'Type of day', axis=alt.Axis(title=None, labels=False, ticks=False)), 
  y = alt.Y('count:Q', title = 'Number of collisions'),
  color= alt.Color('year:O', scale = alt.Scale(domain=[2018, 2020], range=['#ff7f0e', 'steelblue'])),
  column = alt.Column('day_week:N', title='Day of the Week', 
  sort=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], 
  header=alt.Header(titleOrient='bottom', labelOrient='bottom', labelPadding=4))
).transform_calculate(
  year = 'year(datum.CRASH_DATETIME)',
).transform_aggregate(
   count='count()',
   groupby=['year', 'day_week']
)

# paired_bar_chart

We can observe from the previous graph that it allows differentiation between the years for the same day of the week, enabling a comparison before and after COVID. However, we've noticed that for comparing and analyzing whether there have been more accidents on weekdays or weekends, the chart is somewhat limited. Therefore, our next step is to add a slope chart to incorporate this information, which is not being well encoded at the moment.

In [259]:
slope_chart = alt.Chart(frequent_collisions).mark_line(point=True).encode(
  x = alt.X('type_day:O', title = 'Type of day'),
  y = alt.Y('avg:Q', title = 'Number of collisions', axis=alt.Axis(title=None)),
  color= alt.Color('year:O', scale = alt.Scale(domain=[2018, 2020], range=['#ff7f0e', 'steelblue']))
).transform_calculate(
  year = 'year(datum.CRASH_DATETIME)'
).transform_aggregate(
   count='count()',
   groupby=['year', 'day_week', 'type_day']
).transform_aggregate(
    avg = 'mean(count)',
    groupby=['year', 'type_day']
)

# (paired_bar_chart | slope_chart).properties(
#      title='Number of collisions by day of the week and year'
# ).configure_title(anchor='middle').configure_view(stroke='transparent').resolve_scale(y='shared', x='independent', color='shared')

In the preceding bar chart, orange bars represent data from the year 2018, while blue bars represent data from 2020 (before and after COVID, respectively). Examining the length of the bars reveals a consistent trend: on all days of the week, the number of accidents (in total) occurring separately on each day is considerably higher (more than double in all cases) before COVID compared to after.

Moreover, in the right slope chart, where colors correspond to those in the paired bar chart, a decreasing trend is evident in the number of accidents on weekdays versus weekends. Weekends consistently show a lower average number of accidents both before and after COVID. Notably, this difference intensifies before COVID, indicating a more pronounced contrast. However, in 2020, the difference is not as significant.

From this graph, we can infer that the number of traffic accidents has decreased post-COVID, and this reduction in accidents on weekends has followed a similar trend, with a slight decrease in the decline. One possible explanation is the reduced use of both public and private transportation on weekends due to decreased overall activity (work, school, etc.).

### Is there any type of vehicle more prone to participate in accidents?

In [260]:
vehicle_type = pd.DataFrame({'vehicle_type': list(collisions['VEHICLE_TYPE_CODE1'].values) + list(collisions['VEHICLE_TYPE_CODE2'])})
vehicle_type = vehicle_type.groupby('vehicle_type').size().reset_index(name='counts')
most_collisioned = list(vehicle_type.sort_values(by='counts', ascending=False).head(10)['vehicle_type'])
vehicle_type['vehicle_type'] = vehicle_type['vehicle_type'].apply(lambda x: x if x in most_collisioned else 'Others')
vehicle_type = vehicle_type.groupby('vehicle_type').sum('counts').reset_index()
vehicle_type['character'] = vehicle_type['vehicle_type'].apply(lambda x: 'Public' if x in ['Bus', 'Taxi'] else 'Private')
vehicle_type = vehicle_type.sort_values(by='counts', ascending=False).reset_index(drop=True)
vehicle_type.to_csv('../data/vehicle_type.csv', index=False)
vehicle_type.head()

,vehicle_type,counts,character
0,Sedan,92668,Private
1,Station Wagon/Sport Utility Vehicle,70178,Private
2,Taxi,8428,Public
3,Others,6700,Private
4,Pickup,6530,Private


This initial analysis aims to identify the most frequently involved vehicles in accidents. To achieve this, we will employ a bar chart where the number of accidents is represented on the x-axis, and the names of the vehicles are plotted on the y-axis. The choice of a bar chart is intentional, as it allows for a clear visual comparison of accident frequencies across various vehicles. This visualization method optimally presents the names of the vehicles, facilitating a comprehensive understanding of their respective accident counts. 

In [261]:
bar_chart = alt.Chart(vehicle_type).mark_bar().encode(
    x=alt.X('counts:Q', title='Number of collisions'),
    y=alt.Y('vehicle_type:N', 
            sort=list(vehicle_type.loc[vehicle_type['vehicle_type'] != 'Others', 'vehicle_type']) + ['Others'], 
            title='Vehicle Type'),
    color=alt.condition(
        alt.datum.vehicle_type == 'Others',
        alt.value('grey'),
        alt.Color('character:N', legend=alt.Legend(title='Ownership Type'), scale=alt.Scale(domain=['Public', 'Private'], range=['#4daf4a', '#377eb8'])),
    ),  
)

mean_line = alt.Chart(vehicle_type).mark_rule(color='red', strokeWidth=1.5).encode(x = alt.X('mean(counts):Q'))

# (bar_chart + mean_line).properties(
#    title=alt.TitleParams(text='Number of collisions by vehicle type', fontSize=14, subtitle='', offset=20),
#    width=300,
#    height=400
# ).configure_title(anchor='middle').configure_view(stroke='transparent')

The issue with this chart is that it displays the number of accidents for each vehicle type during the given time period. However, it doesn't provide information about whether a particular vehicle is more prone to accidents than another, as we lack data on the total number of each type of vehicle. Nevertheless, it's evident that privately owned cars (tourism vehicles) are involved in the majority of accidents in New York City, significantly surpassing the average. Following this, we notice other vehicles that are frequently involved in accidents, such as taxis, pickups, or bicycles, for example.

### Are there any areas with a larger number of accidents?

In [262]:
geo_collisions = collisions[['LATITUDE', 'LONGITUDE', 'ZIP_CODE', 'BOROUGH']]
geo_collisions = geo_collisions.dropna()
geo_collisions['ZIP_CODE'] = geo_collisions['ZIP_CODE'].astype(str).apply(lambda x: x.split('.')[0])
geo_collisions.to_csv('../data/geo_collisions.csv', index=False)
ny_city_map = alt.topo_feature('../data/ny_city_map.geojson', 'map')

In [263]:
ny_city = alt.Chart(ny_city_map).mark_geoshape(
    fill='lightgray',  
    stroke='white',    
    strokeWidth=1.5    
)                            

The initial idea is to create a point map where each accident is marked as a point on the New York map. This way, we aim to visualize which areas have more points and, consequently, more accidents. Additionally, we have considered that it would be helpful to have a reference for the number of accidents in each borough to scale the visualization. We will also reduce the opacity of the points to better distinguish those that are close together. We should use a categorical color palette since there is no specific order, and it should allow for clear differentiation between the various neighborhoods. Furthermore, we won't include redundant neighborhood information in the bar chart, as it is already encoded with color.

In [264]:
point_map = alt.Chart(geo_collisions).mark_circle(size=1, opacity=0.7).encode(
    latitude='LATITUDE:Q',
    longitude='LONGITUDE:Q',
    color = alt.Color('BOROUGH:N', 
                      scale=alt.Scale(scheme='tableau10'))
)

bar_chart_map = alt.Chart(geo_collisions).mark_bar().encode(
    alt.X('BOROUGH:N', title='Borough', axis=alt.Axis(title=None, labels=False, ticks=False)),
    alt.Y('count():Q', title='Number of Accidents'),
    color = alt.Color('BOROUGH:N', 
                      scale=alt.Scale(scheme='tableau10'),)
)
    
# ((ny_city + point_map).properties(
#     width=380,
#     height=380
# ) | bar_chart_map).properties(
#     title = alt.TitleParams(text='Number of collisions by borough', 
#                             fontSize=18, 
#                             subtitle='', 
#                             offset=25)
# ).configure_title(
#     anchor='middle'
# )

In [265]:
gdf = gpd.read_file('../data/ny_city_map.geojson')
gdf_meters = gdf.to_crs(epsg=3395)
gdf = gdf.merge(geo_collisions.groupby(['ZIP_CODE']).size().reset_index(name='count').rename(columns={'ZIP_CODE': 'postalCode'}), on='postalCode', how='left')
gdf['normalized_by_area_count'] = gdf['count']/(gdf['Shape_Area'] / 1e6)
gdf['log_normalized_by_area_count'] = gdf['normalized_by_area_count'].apply(lambda x: np.log(x) if x > 0 else 0)
gdf.to_csv('../data/gdf.csv', index=False)

We have observed that the point map does not efficiently differentiate areas with more accidents. Due to the high volume of points, they overlap, creating regions of solid color that are challenging to interpret. Therefore, the next step will be to try a choropleth map, which differentiates by PostalCode and encodes the number of accidents in each area with color. One issue we encounter is that larger map regions will generally have more accidents, making it an unfair comparison. To address this, we have decided to normalize the number of accidents in a specific area by the area it occupies. Instead of encoding the raw number of accidents, we will encode the number of accidents per square kilometer to facilitate meaningful comparisons. We will use a Sequential Single-Hue palette for clarity.

In [266]:
# gdf.sort_values(by='normalized_by_area_count', ascending=False)[['postalCode', 'normalized_by_area_count']].head(20)

In [267]:
choropleth_map = alt.Chart(gdf).mark_geoshape().encode(
    alt.Color('normalized_by_area_count:Q',
              # title='Number of Accidents per km^2', 
              scale=alt.Scale(scheme='lightorange', domain=[0, 40]), 
              legend = None),
).properties(
    width=500,
    height=450,
)

borough_names = alt.Chart(geo_collisions).mark_text(fontWeight='bold', fontSize=11, color='black').encode(
    latitude='mean_lat:Q',
    longitude='mean_long:Q',
    text='BOROUGH:N',
).transform_aggregate(
    mean_lat='mean(LATITUDE)',
    mean_long='mean(LONGITUDE)',
    groupby=['BOROUGH']
)

# (choropleth_map + borough_names).properties(
#      title=alt.TitleParams(text='Number of Collisions by PostalCode', fontSize=16, subtitle='', offset=20),
# ).configure_title(anchor='middle')

In [268]:
collisions.columns

Index(['BOROUGH', 'ZIP_CODE', 'LATITUDE', 'LONGITUDE', 'TOTAL_INJURED',
       'TOTAL_KILLED', 'PEDESTRIANS_INJURED', 'PEDESTRIANS_KILLED',
       'CYCLIST_INJURED', 'CYCLIST_KILLED', 'MOTORIST_INJURED',
       'MOTORIST_KILLED', 'CONTRIBUTING_FACTOR_VEHICLE1',
       'CONTRIBUTING_FACTOR_VEHICLE2', 'CONTRIBUTING_FACTOR_VEHICLE3',
       'CONTRIBUTING_FACTOR_VEHICLE4', 'CONTRIBUTING_FACTOR_VEHICLE5',
       'VEHICLE_TYPE_CODE1', 'VEHICLE_TYPE_CODE2', 'CRASH_DATETIME',
       'day_week', 'type_day'],
      dtype='object')

In [269]:
death_rate = collisions[['TOTAL_INJURED','TOTAL_KILLED', 'PEDESTRIANS_INJURED', 'PEDESTRIANS_KILLED',
                         'CYCLIST_INJURED', 'CYCLIST_KILLED', 'MOTORIST_INJURED', 'MOTORIST_KILLED']]
death_rate = death_rate.dropna()

In [270]:
death_rate.head()

,TOTAL_INJURED,TOTAL_KILLED,PEDESTRIANS_INJURED,PEDESTRIANS_KILLED,CYCLIST_INJURED,CYCLIST_KILLED,MOTORIST_INJURED,MOTORIST_KILLED
0,0.0,1.0,0,0,0,0,0,1
1,0.0,0.0,0,0,0,0,0,0
2,0.0,0.0,0,0,0,0,0,0
3,1.0,0.0,0,0,0,0,1,0
4,0.0,1.0,0,0,0,1,0,0
